In [ ]:
def book_pattern_vis(id, chunksize=512, ybottom=0):
    #We need to identify the age of the book to use the correct model. Extract year from metadata
    import csv
    with open('/scratch/project_2007227/genre_data/ecco_metadata.tsv', 'r') as tsv_file:
        reader = csv.DictReader(tsv_file, delimiter='\t')
        for row in reader:
            if row['document_id_octavo'] == str(id):
                year = int(row['publication_year'])
    
    #if the book is from earlier era than 1720, gives erros since we don't have model from that era.
    if year < 1720:
        print('Publication year of the document is not included in the models, please try another document.')
        return True
    
    #if the book is ok, we need to first preprocess the document for the model
    from contextualized_topic_models.models.ctm import ZeroShotTM
    from contextualized_topic_models.utils.data_preparation import TopicModelDataPreparation
    from contextualized_topic_models.utils.preprocessing import WhiteSpacePreprocessingStopwords
    import nltk
    from nltk.corpus import stopwords as stop_words
    
    
    #First divide the text into chunks
    with open('/scratch/project_2007227/genre_data/ecco_source/'+str(id)+'.txt', 'r') as f:
        lines = f.readlines()
    
    num_lines = len(lines)
    text = " ".join(lines)
    new_text = '\n'.join([text[i:i+chunksize] for i in range(0, len(text), chunksize)])
    
    with open('/scratch/project_2007227/genre_data/chunk512/'+str(id)+'512.txt', 'w') as f:
        f.write(new_text)
              
    nltk.download('stopwords')
    text_file = '/scratch/project_2007227/genre_data/chunk512/'+str(id)+'512.txt'
    documents = [line.strip() for line in open(text_file, encoding="utf-8").readlines()[0:num_lines]]

    stopwords = list(stop_words.words("english"))

    sp = WhiteSpacePreprocessingStopwords(documents, stopwords_list=stopwords)
    preprocessed_documents, unpreprocessed_corpus, vocab, retained_indices = sp.preprocess()
    
    import pickle
    tp = TopicModelDataPreparation("paraphrase-multilingual-mpnet-base-v2")
    processed_data= tp.fit(text_for_contextual=unpreprocessed_corpus, text_for_bow=preprocessed_documents)
    
    #Nextly we fetch the model that corresponds to the publication year
    modelnumber = ((year // 10) % 10)-2
    
    with open('/scratch/project_2007227/genre_data/Digital-Humanities-Project-Genres-2023/src/tp_models/tp_model'+str(modelnumber)+'.pickle', 'rb') as tpmodel:
        tpmodeli = pickle.load(tpmodel)
    ctm = ZeroShotTM(bow_size=len(tpmodeli.vocab), contextual_size=768, n_components=10, num_epochs=20)
    ctm.load("/scratch/project_2007227/genre_data/Digital-Humanities-Project-Genres-2023/src/ctm_models"+str(modelnumber)+"/contextualized_topic_model_nc_10_tpm_0.0_tpv_0.9_hs_prodLDA_ac_(100, 100)_do_softplus_lr_0.2_mo_0.002_rp_0.99",
                                                                                                          epoch=19)
    pred_model = ctm.get_thetas(processed_data, n_samples=20) 
    
    import matplotlib.pyplot as plt
    plt.plot(pred_model, '.')
    plt.ylim((ybottom,1))
    plt.title('Sequential genre prediction for document '+str(id)+' in with 512 token chunks')

In [ ]:
book_pattern_vis(1307301100)
#Currently the method doesn't support ids starting with 0, to address this feel free to modify str<->int in the code, 